In [2]:
import requests
import csv
import datetime
import pandas as pd
import os
import numpy as np

# Получение токена

In [ ]:
# Данные для OAuth2
client_id = ''
client_secret = ''
redirect_uri = 'https://example.com/page'
authorization_code = ''  # Ваш authorization code

# URL для получения токена
token_url = 'https://hh.ru/oauth/token'

# Параметры запроса
data = {
    'grant_type': 'authorization_code',
    'client_id': client_id,
    'client_secret': client_secret,
    'code': authorization_code,
    'redirect_uri': redirect_uri
}
   

# Отправка POST-запроса на получение токена
response = requests.post(token_url, data=data)
tokens = response.json()

# Проверка, если запрос успешен и вывод токена
if 'access_token' in tokens:
    access_token = tokens['access_token']
    print(f"Access Token: {access_token}")
else:
   print("Error obtaining access token:", tokens)

 # Сбор данных

In [ ]:
# Токен доступа
access_token = ''  # Замените на полученный access token

# URL для поиска вакансий
vacancies_url = 'https://api.hh.ru/vacancies'

# Заголовки для запроса
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Authorization': f'Bearer {access_token}'
}

# Функция для генерации списка месячных интервалов за год
def generate_date_ranges(start_date, end_date):
    date_ranges = []
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + datetime.timedelta(days=30)
        if next_date > end_date:
            next_date = end_date
        date_ranges.append((current_date, next_date))
        current_date = next_date
    return date_ranges

# Генерация диапазонов дат за последний год
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=365)
date_ranges = generate_date_ranges(start_date, end_date)

# Список текстовых параметров
text_parameters = [
    '1с OR 1с программист OR консультант 1с OR 1с специалист',
    'python OR питон OR python разработчик OR питонист',
    'c# OR c sharp OR разработчик c# OR c sharp программист',
    'c++ OR c plus plus OR разработчик c++ OR c++ программист',
    'c OR язык си OR системное программирование OR си программист',
    'java OR разработчик java OR ява OR программирование на java',
    'full-stack OR full stack OR full-stack разработчик OR full stack инженер',
    'backend OR back-end OR разработчик backend OR бэкенд разработка',
    'frontend OR front-end OR разработчик frontend OR веб-разработчик',
    'developer OR разработчик OR программист OR инженер-программист',
    'системный аналитик OR системный анализ OR аналитик систем OR системный архитектор',
    'системное программирование OR системное разработка OR системный программист OR системный инженер',
    'data analyst OR дата аналитик OR дата анализ OR аналитик данных OR аналитика данных',
    'machine learning OR ml',
    'big data OR большие данные OR обработка больших данных OR анализ больших данных',
    'devops OR девопс OR devops инженер OR devops специалист',
    'cloud computing OR облачные вычисления OR облачные сервисы OR облачные технологии',
    'blockchain OR блокчейн OR блокчейн разработчик OR технология блокчейн',
    'cybersecurity OR кибербезопасность OR безопасность сети OR специалист по информационной безопасности',
    'ui/ux design OR дизайн интерфейса OR пользовательский интерфейс OR дизайнер ui/ux',
    'mobile development OR мобильная разработка OR разработчик мобильных приложений OR мобильный разработчик',
    'embedded systems OR встраиваемые системы OR встраиваемое программирование OR embedded разработка',
    'iot OR интернет вещей OR разработка для интернета вещей OR специалист по IoT',
    'game development OR разработка игр OR геймдев OR геймдизайнер',
    'quality assurance OR аналитик тестирования OR тестировщик OR qa инженер',
    'project management OR управление проектами OR проектный менеджмент OR руководитель проектов',
    'business analysis OR бизнес-анализ OR аналитик бизнес-процессов OR бизнес-консультант',
    'technical support OR техническая поддержка OR it поддержка OR специалист по технической поддержке',
    'тестировщик OR AB-тесты OR ab тестирование OR инженер по тестированию'
]

# Обработка всех текстовых параметров последовательно
for idx, text_param in enumerate(text_parameters):
    csv_filename = f'vacancies_{idx + 1}.csv'
    # Открываем CSV файл для записи в UTF-8
    with open(csv_filename, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file, delimiter=';')
        # Записываем заголовок столбцов
        writer.writerow(['название_вакансии', 'зарплата', 'название_компании', 'опыт работы', 'город', 'регион_область', 'требования_обязанности'])
        
        for date_range in date_ranges:
            params = {
                'text': text_param,
                'area': [1, 113],  # Москва и Россия
                'per_page': 100,  # Количество вакансий на странице
                'page': 0,  # Номер страницы, начнем с 0
                'date_from': date_range[0].strftime('%Y-%m-%d'),
                'date_to': date_range[1].strftime('%Y-%m-%d')
            }

            while True:
                # Выполнение запроса к API
                response = requests.get(vacancies_url, headers=headers, params=params)
                data = response.json()

                # Извлечение данных и запись в CSV
                if 'items' in data:
                    vacancies = data['items']
                    for vacancy in vacancies:
                        name = vacancy['name']
                        employer = vacancy['employer']['name'] if 'employer' in vacancy else 'не указана'
                        
                        salary = vacancy.get('salary', {})
                        if salary:
                            if salary.get('from') and salary.get('to'):
                                average_salary = (salary['from'] + salary['to']) / 2
                                salary_str = f'{average_salary:.2f}'
                            elif salary.get('from'):
                                salary_str = f"{salary['from']}"
                            elif salary.get('to'):
                                salary_str = f"{salary['to']}"
                            else:
                                salary_str = 'не указана'
                        else:
                            salary_str = 'не указана'

                        experience = vacancy.get('experience', {}).get('name', 'не указано')

                        address = vacancy.get('address')
                        city = address['city'] if address and 'city' in address else 'не указано'
                        region = vacancy.get('area', {}).get('name', 'не указано')

                        requirement = vacancy.get('snippet', {}).get('requirement', 'не указано')
                        responsibility = vacancy.get('snippet', {}).get('responsibility', 'не указано')
                        requirements_responsibilities = f"{requirement} / {responsibility}"

                        writer.writerow([name, salary_str, employer, experience, city, region, requirements_responsibilities])

                    # Проверка, есть ли еще страницы
                    if params['page'] < data['pages'] - 1:
                        params['page'] += 1  # Переход на следующую страницу
                    else:
                        break  # Все страницы обработаны
                else:
                    print("Error:", data)
                    break

    print(f"Данные для текстового параметра '{text_param}' записаны в файл {csv_filename}")




# Очистка данных

In [ ]:
# Функция для удаления дубликатов из CSV файла
def remove_duplicates(file_path):
    # Читаем CSV файл с указанием разделителя ;
    df = pd.read_csv(file_path, delimiter=';')
    # Удаляем дубликаты и пустые строки
    df.drop_duplicates(subset=df.columns.difference(['город', 'регион_область']), inplace=True)
    df.dropna(inplace=True)
    # Записываем результат обратно в CSV файл с кодировкой utf-8-sig
    df.to_csv(file_path, index=False, sep=';', encoding='utf-8-sig')
    print(f"Дубликаты удалены из файла {file_path}")

# Обрабатываем все 29 таблиц
for idx in range(29):
    csv_filename = f'vacancies_{idx + 1}.csv'
    remove_duplicates(csv_filename)


In [ ]:
# Функция для обработки файла
def process_file(file_path):
    # Читаем CSV файл с указанием разделителя ;
    df = pd.read_csv(file_path, delimiter=';')
    
    # Приводим все текстовые значения к нижнему регистру
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    
    # Обрабатываем столбец зарплата и создаем два новых столбца
    if 'зарплата' in df.columns:
        # Создаем столбец с числовыми значениями зарплаты
        df['зарплата_число'] = df['зарплата'].apply(
            lambda x: int(float(x)) if isinstance(x, str) and x.replace('.', '', 1).isdigit() else ''
        )
        # Создаем столбец с текстовым значением "не указана"
        df['зарплата_текст'] = df['зарплата'].apply(
            lambda x: x if isinstance(x, str) and not x.replace('.', '', 1).isdigit() else ''
        )
        # Удаляем исходный столбец зарплата
        df.drop(columns=['зарплата'], inplace=True)
    
    # Удаляем столбец "город"
    if 'город' in df.columns:
        df.drop(columns=['город'], inplace=True)
    
    # Переименовываем столбец "регион_область" в "местоположение" 
    if 'регион_область' in df.columns:
        df.rename(columns={'регион_область': 'местоположение'}, inplace=True)
    
    # Заменяем значения "None / None" на "не указаны" в последнем столбце
    df.iloc[:, -1] = df.iloc[:, -1].fillna('не указаны').replace('None / None', 'не указаны')
    
    # Удаляем <highlighttext> и </highlighttext> из последнего столбца
    df.iloc[:, -1] = df.iloc[:, -1].str.replace('<highlighttext>', '').str.replace('</highlighttext>', '')
    
    # Записываем результат обратно в CSV файл с кодировкой utf-8-sig
    df.to_csv(file_path, index=False, sep=';', encoding='utf-8-sig')
    print(f"Файл {file_path} успешно обработан")

# Обрабатываем все 29 таблиц
for idx in range(29):
    csv_filename = f'vacancies_{idx + 1}.csv'
    if os.path.exists(csv_filename):
        process_file(csv_filename)


In [4]:
# Объединяем все файлы в один DataFrame
combined_df = pd.DataFrame()

for idx in range(29):
    csv_filename = f'vacancies_{idx + 1}.csv'
    if os.path.exists(csv_filename):
        df = pd.read_csv(csv_filename, delimiter=';')
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Удаляем дубликаты
combined_df = combined_df.drop_duplicates(subset=combined_df.columns.difference(['город', 'регион_область']))

# Удаляем пустые строки
combined_df.dropna(how='all', inplace=True)

# Сохраняем объединенный файл
combined_df.to_csv('combined_vacancies.csv', index=False, sep=';', encoding='utf-8-sig')

# Выводим количество строк
print(f"Количество строк в объединенном файле: {combined_df.shape[0]}")


Количество строк в объединенном файле: 68660
